Данный скрипт позволяет найти параметры эллипсоида, по которому движется центр масс лопатки, методами градиентного спуска (рекомендуется к применению) и методом Ньютона (не рекомендуется из-за неусточйчивости).

In [ ]:
import numpy as np
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
from numpy import linalg as LA

In [36]:
#Все маркеры
markers = ['RAH', 'RPH', 'LAH', 'LPH', 'C7', 'T8', 'IJ', 'PX','RSC', 'RAC','LSC', 'LAC', 'RAI', 'RPC',
           'RTS', 'RAA', 'LAI', 'LTS', 'LAA', 'LPC', 'REM', 'REL', 'LEM', 'LEL', 'RRS', 'RUS', 'LRS', 'LUS', 'LSJ', 'RSJ']
#Маркеры, условно неподвижные относительно тела
stablemarkers = ['T8', 'IJ', 'PX', 'C7']
#Маркеры лопатки
markersscap = ['RTS', 'RAI', 'RAA']
#Маркер грудины
markersternum = ['IJ'] 
#Хранит названия колонок в списке
cols = []
stablecols = []
scapulacols = []
sternumcols = []
#Хранит названия колонок в строке
csvcols = ''
for i in markers:
    cols.append(i + 'x')
    cols.append(i + 'y')
    cols.append(i + 'z')
    csvcols += cols[-3] + '\t' + cols[-2] + '\t' + cols[-1] + '\t'
for i in stablemarkers:
    stablecols.append(i + 'x')
    stablecols.append(i + 'y')
    stablecols.append(i + 'z')
for i in markersscap:
    scapulacols.append(i + 'x')
    scapulacols.append(i + 'y')
    scapulacols.append(i + 'z')
for i in markersternum:
    sternumcols.append(i + 'x')
    sternumcols.append(i + 'y')
    sternumcols.append(i + 'z')

In [1]:
#Файлы для анализа
files = ['1.trc', '2.trc','3.trc','4.trc','5.trc','6.trc','7.trc','8.trc','9.trc','9_combined.trc','9_left.trc','9_right.trc',
'10.trc', '11.trc','12.trc','12_shrugging.trc','13.trc','14.trc','14_depression.trc', '14_elevation.trc', 'very long static 14.trc']
#Тут можно задать префикс/постфикс для путей к файлам
for i in range(len(files)):
    files[i] = 'smoothed_trc\\' + files[i][0:-4] + '_shifted_smoothed.trc'
    
for file in files:
    datalines = []
    with open (file, 'r') as fp:
        datalines = fp.readlines()
        infolines = datalines[0:5]
        #infolines[1].replace('NumFrames', 'NumFrame', 1)
        #infolines[1].replace('NumMarkers', 'NumMarker', 1)
        datalines = datalines[5:-1]
    textdatalines = ''
    textinfodatalines = ''
    for i in datalines:
        textdatalines += i + '\n'
    #df хранит данные оригинального файла
    df = pd.read_csv(StringIO(textdatalines), sep = '\t', names = ['Frame#', 'Time'] + cols)
    #Coordinate of Scapula x, y, z
    csx = np.zeros_like(df[markersscap[0] + 'x'].values)
    csy = np.zeros_like(csx)
    csz = np.zeros_like(csx)
    for i in markersscap:
        csx += np.array(df[i + 'x'])
        csy += np.array(df[i + 'y'])
        csz += np.array(df[i + 'z'])
    csx /= len(markersscap)
    csy /= len(markersscap)
    csz /= len(markersscap)
    
    #Still Subject coordinates in laboratory frame. Describe the movement of the point, which is still relatively the body
    SSx = np.zeros(len(df.index))
    SSy = np.zeros(len(df.index))
    SSz = np.zeros(len(df.index))

    for i in stablecols:
        if 'x' in i:
            SSx += np.array(df[i])
        elif 'y' in i:
            SSy += np.array(df[i])
        elif 'z' in i:
            SSz += np.array(df[i])

    SSx /= len(stablecols) 
    SSy /= len(stablecols)
    SSz /= len(stablecols)
    
    #critical std differentiates still markers between moving ones. when the std is greater than 3sigma for a relatively
    #still point, a marker must be recognised as a moving one. sigma is calculated as 6 maximal std of still subject's
    #coordinates
    critstd = ' %.2f' %(6 * max(SSx.std(), SSy.std(), SSz.std()))
    
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(projection='3d')
    ax.grid(color='#000000', alpha=0.1, linestyle='-', linewidth=1, which='major')
    ax.grid(color='#000000', alpha=0.1, linestyle='-', linewidth=0.5, which='minor')
    plt.title('Scapular movement in ')
    stdx = ' %.2f'%(csx.std())
    stdy = ' %.2f'%(csy.std())
    stdz = ' %.2f'%(csz.std())
    
    plt.title('Scapular movement in ' + file[13:-4] + 'std x/y/z/crit = ' + stdx + stdy + stdz + critstd + 'mm')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.scatter(csx, csy, csz)
    
    #Сохранение картинок
    plt.savefig('ellipsoid\\' + file[13:-4] + '.png')
    plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'smoothed_trc\\1_shifted_smoothed.trc'

In [73]:
#Минимизируемая функция
def f(x, y, z, coord):
    ans = 0.0
    x0 = coord[0]
    y0 = coord[1]
    z0 = coord[2]
    a = coord[3]
    b = coord[4]
    c = coord[5]
    for i in range(len(x)):
        ans += ((x[i] - x0)** 2/a**2 + (y[i] - y0)** 2/b**2 + (z[i] - z0)** 2 / c ** 2 - 1) ** 2
    return ans
#Градиент минимизируемой функции
def gradf(x, y, z, coord):
    x0 = coord[0]
    y0 = coord[1]
    z0 = coord[2]
    a = coord[3]
    b = coord[4]
    c = coord[5]
    ans = np.zeros(6)
    for i in range(len(x)):
        fi = (x[i] - x0)**2/a**2 + (y[i] - y0)** 2/b**2 + (z[i] - z0)** 2/c**2 - 1
        ans[0] += fi * (x0 - x[i])
        ans[1] += fi * (y0 - y[i])
        ans[2] += fi * (z0 - z[i])
        ans[3] += (x[i]-x0) ** 2 * fi
        ans[4] += (y[i]-y0) ** 2 * fi
        ans[5] += (z[i]-z0) ** 2 * fi 
    ans[0] *= 4/a**2
    ans[1] *= 4/b**2
    ans[2] *= 4/c**2
    ans[3] *= -4/a**3
    ans[4] *= -4/b**3
    ans[5] *= -4/c**3
    return ans
#Якобиан минимизируемой функции
def jacobf(x, y, z, coord):
    x0 = coord[0]
    y0 = coord[1]
    z0 = coord[2]
    a = coord[3]
    b = coord[4]
    c = coord[5]
    ans = np.zeros((6,6))
    for i in range(len(x)):
        fi = (x[i] - x0)**2/a**2 + (y[i] - y0)** 2/b**2 + (z[i] - z0)** 2/c**2 - 1
        dx = x0 - x[i]
        dy = y0 - y[i]
        dz = z0 - z[i]
        
        ans[0][0] += 2 / a ** 2 * dx ** 2 + fi
        ans[1][1] += 2 / b ** 2 * dy ** 2 + fi
        ans[2][2] += 2 / c ** 2 * dz ** 2 + fi
        ans[3][3] += 12 / a ** 4 * fi * dx ** 2 + 8 / a ** 6 * dx ** 4
        ans[4][4] += 12 / b ** 4 * fi * dy ** 2 + 8 / b ** 6 * dy ** 4
        ans[5][5] += 12 / c ** 4 * fi * dz ** 2 + 8 / c ** 6 * dz ** 4
        
        ans[0][1] += dx * dy
        ans[0][2] += dx * dz
        ans[1][2] += dy * dz
        ans[0][3] += -8 / a**3 * fi * dx - 8 / a**5 * dx ** 3 
        ans[0][4] += dx * dy ** 2 
        ans[0][5] += dx * dz ** 2 
        ans[1][3] += dy * dx ** 2 
        ans[1][4] += -8 / b ** 3 * fi * dy - 8 / b ** 5 * dy ** 3
        ans[1][5] += dy * dz ** 2 
        ans[2][3] += dz * dx ** 2 
        ans[2][4] += dz * dy ** 2
        ans[2][5] += -8 / c ** 3 * fi * dz - 8 / c ** 5 * dz ** 3
        ans[3][4] += dx **2 * dy ** 2
        ans[3][5] += dx ** 2 * dz ** 2
        ans[4][5] += dy ** 2 * dz ** 2
          
    ans[0][0] *= 4 / a ** 2
    ans[1][1] *= 4 / b ** 2
    ans[2][2] *= 4 / c ** 2
    ans[0][1] *= 8 / a **2 / b ** 2
    ans[0][2] *= 8 / a **2 / c ** 2
    ans[1][2] *= 8 / b **2 / c ** 2
    ans[0][4] *= -8 / a**2 / b ** 3
    ans[0][5] *= -8 / a**2 / c ** 3
    ans[1][3] *= -8 / b**2 / a ** 3
    ans[1][5] *= -8 / b ** 2 / c ** 3
    ans[2][3] *= -8 / c ** 2 / a ** 3
    ans[2][4] *= -8 / c ** 2 / b ** 3
    ans[3][4] *= 8 / a ** 3 / b ** 3
    ans[3][5] *= 8 / a ** 3 / c ** 3
    ans[4][5] *= 8 / b ** 3 / c ** 3
    for i in range(6):
        for j in range(0, i):
            ans[i][j] = ans[j][i]
    return ans 

In [118]:
files = ['9_combined.trc','9_right.trc', '12.trc','14_depression.trc']
files = ['14_elevation.trc']
files = ['14_depression.trc']
#Файлы для обработки
files = ['14.trc']
#Добавление префикса/постфикса
for i in range(len(files)):
    files[i] = 'smoothed_trc\\' + files[i][0:-4] + '_shifted_smoothed.trc'

for file in files:
        datalines = []
        with open (file, 'r') as fp:
            datalines = fp.readlines()
            infolines = datalines[0:5]
            datalines = datalines[5:-1]
        textdatalines = ''
        textinfodatalines = ''
        for i in datalines:
            textdatalines += i + '\n'
        #df хранит данные оригинального файла
        df = pd.read_csv(StringIO(textdatalines), sep = '\t', names = ['Frame#', 'Time'] + cols)
        
        df1 = df.copy()
        #Sternum coordinates
        STx = np.zeros(len(df.index))
        STy = np.zeros(len(df.index))
        STz = np.zeros(len(df.index))

        for i in sternumcols:
            if 'x' in i:
                STx += np.array(df1[i])
            elif 'y' in i:
                STy += np.array(df1[i])
            elif 'z' in i:
                STz += np.array(df1[i])

        STx /= len(sternumcols) 
        STy /= len(sternumcols)
        STz /= len(sternumcols)

        for i in cols:
            if 'x' in i:
                for j in range(len(df1[i].values)):
                    df1[i].values[j] -= STx[j]
                    #df1[i].values[j] += 10
            elif 'y' in i:
                for j in range(len(df1[i].values)):
                    df1[i].values[j] -= STy[j]
                    #df1[i].values[j] += -26
            elif 'z' in i:
                for j in range(len(df1[i].values)):
                    df1[i].values[j] -= STz[j]
                    df1[i].values[j] += -22.7
        
        #Coordinate of Scapula x, y, z
        csx = np.zeros_like(df[markersscap[0] + 'x'].values)
        csy = np.zeros_like(csx)
        csz = np.zeros_like(csx)
        for i in markersscap:
            csx += np.array(df[i + 'x'])
            csy += np.array(df[i + 'y'])
            csz += np.array(df[i + 'z'])
        csx /= len(markersscap)
        csy /= len(markersscap)
        csz /= len(markersscap)

        #x0, y0, z0, a, b, c
        #Начальное приближение для переменных выше
        coord = [-45, -5, 55, 80, 250, 80]
        #Постонный шаг метода градиентного спуска
        tau = 0.1
        #Норма градиента (инициализирована очень большим значением)
        normg = 1e10
        gradcalc = gradf(csx, csy, csz, coord)
        #значение функции
        fcalc = 1e10
        
        #fcalc = f(csx, csy, csz, coord)
        #while LA.norm(gradcalc) < normg:
        #Условие останова по норме градиента
        while LA.norm(gradcalc) > 0.1:
        #while f(csx, csy, csz, coord) < fcalc:
            fcalc = f(csx, csy, csz, coord)
            normg = LA.norm(gradcalc)
            coord -= tau * gradcalc
            gradcalc = gradf(csx, csy, csz, coord)
            #
            #print(file, 'fcalc = ', fcalc)
        print(file, 'fcalc = ', fcalc)
        print(file, ' ', coord)
        
        #Это отладочный код, которым я проверял работоспособность метода Ньютона 
        """
        #H = LA.inv(jacobf(csx, csy, csz, coord))
        #gradcalc = gradf(csx, csy, csz, coord)
        #test
        csx = [1, 0, 0, 0.5**0.5, 0.5**0.5, 0]
        csy = [0, 1, 0, 0.5**0.5, 0, 0.5**0.5]
        csz = [0, 0, 1, 0, 0.5**0.5, 0.5**0.5]
        coord = [0.01,-0.02,0,1.08,1,1.0]
        print(jacobf(csx, csy, csz, coord))
        for i in range(50):
            coord -= LA.inv(jacobf(csx, csy, csz, coord)).dot(gradf(csx, csy, csz,coord))
            
        fcalc = f(csx, csy, csz, coord)
        print(file, 'fcalc = ', fcalc)
        """

smoothed_trc\14_shifted_smoothed.trc fcalc =  2.924940477488844
smoothed_trc\14_shifted_smoothed.trc   [-100.56267729   98.779671     95.81935976  118.4225645   394.94500565
  132.7925609 ]
